In [5]:
import os
from src.d01_ana.analysis import *
from src.d00_utils.helper import chunks, flatten

def compute_populism_score(dfs, dictionary):

    number_documents = len(dfs)

    dfs['score_pop'] = dfs.apply(lambda row: compute_pop_score(row.hits_pop, row['len'], dictionary, number_documents), axis=1)

    dfs['all_counter'] = dfs.apply(lambda row: {**row.volk_counter, **row.elite_counter}, axis=1)
    dfs['score'] = dfs.apply(lambda row: compute_score_from_counts(row.all_counter, row['len'], dictionary, number_documents), axis=1)
    dfs['score_volk'] = dfs.apply(lambda row: compute_score_from_counts(row.volk_counter, row['len'], dictionary, number_documents), axis=1)
    dfs['score_elite'] = dfs.apply(lambda row: compute_score_from_counts(row.elite_counter, row['len'], dictionary, number_documents), axis=1)

    dfs['score_pop_sum'] = dfs.apply(lambda row: compute_score_sum(row.score_pop), axis=1)

    return dfs


def content_analysis(directory, party='all', sample=None):

    if os.path.isdir(directory):
        print('Directory already exists.')
        return
    # party = 'all'

    # lemmatizer = GermaLemma()
    # nlp = spacy.load("de_core_news_lg")

    doc_labels = load_data(party)
    if sample:
        doc_labels = random.sample(doc_labels, sample)

    print('Number of documents: {}'.format(len(doc_labels)))

    print(f'Beginning Content Analysis with parameters: \n Party: {party}')

    c = ContentAnalysis('de_core_news_lg', 'dict', directory=directory)

    for i, batch in enumerate(chunks(doc_labels, 5000)):
        # print(f'Content Analysis on batch:{i+1}')
        res = []
        for label in tqdm(batch):
            res.append(c.analyze(label, gendocs(label)))
        df = pd.DataFrame(res)
        df.to_csv(f'res_ca/{directory}/res_ca_{party}_{i}.csv')

    print(f'Content Analysis complete. \nResults saved in {directory}/...')


def discourse_analysis(directory, party=None, iter=1, sample=None, **kwargs):

    logging.basicConfig(filename='w2v.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    if os.path.isdir(directory):
        print('Directory already exists.')
        return

    Path(f'res_da/{directory}/').mkdir(parents=False, exist_ok=False)

    doc_labels = load_data(party)
    if sample:
        doc_labels = random.sample(doc_labels, sample)

    print('Number of documents: {}'.format(len(doc_labels)))
    sentences = MyCorpus(doc_labels)
    print(f'Beginning Discourse Analysis with parameters: \n{kwargs}')

    # model params
    model = Word2Vec(
    alpha=0.0025, min_alpha=0.00001, vector_size=300, epochs=300, seed=42, workers=8, hashfxn=hash, sorted_vocab=1, sg=1, hs=1, negative=0, sample=1e-4, **kwargs)

    model.build_vocab(sentences)

    # intersect
    pre = KeyedVectors.load('embeddings/wiki.model')
    res = intersect(pre, model)
    del pre

    model.wv.add_vectors(range(model.wv.vectors.shape[0]), res, replace=True)

    # ASSERT?

    total_examples = model.corpus_count

    for i in range(iter):
        try:
            seeds = random.choices(range(1_000_000), k=iter)
            seed = seeds[i]
            print(f'Seed in iteration {i}: {seed}')
            model.seed = seed
            loss_logger = LossLogger(party, i, directory)

            # train
            model.train(sentences, total_examples=total_examples, epochs=model.epochs, compute_loss=True, callbacks=[loss_logger])

        except EndOfTraining:
            print(f'End of Iteration: {i}')

    print(f'Discourse Analysis complete. \nResults saved in {directory}/...')



In [6]:
%env PYTHONHASHSEED=0


env: PYTHONHASHSEED=0


In [7]:
df = load_results_content_analysis('res_ca/results_ca')



In [8]:
def compute_from_counts(counts, doclen, dictionary, number_docs, idf_weight):
    scores = []
    for term, n in counts.items():
        score = compute_idf(term, dictionary, number_docs)
        scores.append((score**idf_weight) * n)
        # print(term, n, score)
    res = sum(scores) / log(doclen+10, 10)
    # res = sum(scores)
    return res


def compute(row, dictionary, number_documents, idf_weight):
    if row['pop'] == True:
        volk = compute_from_counts(row.volk_counter, row.len, dictionary, number_documents, idf_weight)
        elite = compute_from_counts(row.elite_counter, row.len, dictionary, number_documents, idf_weight)
        attr = compute_from_counts(row.attr_counter, row.len, dictionary, number_documents, idf_weight)
    else:
        volk = 0.0
        elite = 0.0
        attr = 0.0
    return (volk, elite, attr)

def scoring(df, dictionary, idf_weight):
    number_documents = len(df)
    df['scores'] = df.apply(lambda row: compute(row, dictionary, number_documents, idf_weight), axis=1)

    # seperate into columns:
    df['score'] = df.apply(lambda row: sum(row.scores), axis=1)
    df['score_volk'] = df.apply(lambda row: row.scores[0], axis=1)
    df['score_elite'] = df.apply(lambda row: row.scores[1], axis=1)
    df['score_attr'] = df.apply(lambda row: row.scores[2], axis=1)



    return df


In [9]:
dictionary = pickle.load(open('plenar_dict.pkl', 'rb'))
df = scoring(df, dictionary, 2)
df.groupby('party').mean()


,len,pop,volk,elite,num_pop,score,score_volk,score_elite,score_attr
party,,,,,,,,,
AfD,630.831473,0.175970,1.836257,1.676768,0.249867,900.496319,294.616369,286.205363,319.674587
CDU,947.486769,0.082582,1.531404,1.302850,0.105263,342.751632,121.141060,104.147527,117.463046
CSU,889.292386,0.082810,1.476845,1.209184,0.106750,347.767240,121.072424,117.101354,109.593463
DIE GRÜNEN,736.441710,0.086966,1.096976,1.543900,0.118874,387.127311,105.402887,121.677675,160.046750
DIE LINKE,732.769249,0.119877,1.522151,1.674722,0.169865,602.162787,181.372669,175.801380,244.988738
Die blaue Partei,515.193277,0.201681,1.537815,1.781513,0.277311,984.997167,197.578002,406.351062,381.068103
FDP,600.107328,0.099250,1.163301,1.385459,0.130987,410.940605,120.524161,139.855647,150.560798
Parteilos,722.244898,0.204082,1.061224,1.265306,0.224490,886.565654,193.679874,243.812021,449.073759
SPD,864.437611,0.082145,1.469846,1.213607,0.105615,346.687885,117.495788,103.013282,126.178816


In [10]:
# EVALUATE!!!
df.sort_values('score', ascending=False)[['party', 'doc']][:500]


,party,doc
date,,
2016-03-16,DIE LINKE,plenar_002882
2014-11-27,DIE LINKE,plenar_011596
2018-04-19,DIE LINKE,plenar_020482
2016-11-23,DIE LINKE,plenar_002889
2019-09-27,SPD,plenar_001357
...,...,...
2016-10-20,CSU,plenar_001025
2015-04-24,CDU,plenar_022421
2015-01-15,SPD,plenar_016801


In [28]:
import statsmodels.api as sm
df_reg = df.copy()
# change daterange?
df_reg = df_reg.loc["2018-01-01":"2020-01-01"]
# df_reg = df_reg.loc[df['pop'] == True]
reg = df_reg[['typ', 'opp', 'date', 'score', 'party']].dropna()

def sommerpause(row):
    if 7 < row.month < 9:
        return "sommerpause"
    else:
        return "keine_sommerpause"

reg['year'] = reg['date'].dt.strftime('%y')
reg['year'] = reg['year'].astype('int')
reg['year'] = reg['year'] - 13
reg['month'] = reg['date'].dt.strftime('%m')
reg['month'] = reg['month'].astype('int')
reg['is_summer'] = reg.apply(lambda row: sommerpause(row), axis=1)

sm.add_constant(reg)

res = sm.Poisson.from_formula("score ~ C(party, Treatment('CDU')) + year", reg, missing='drop').fit()
# res = sm.OLS.from_formula("score ~ C(opp, Treatment('not_opp'))", reg, missing='drop').fit()
# res = sm.Poisson.from_formula("score ~ C(party, Treatment('CDU')) + date * C(party, Treatment('CDU'))", reg, missing='drop').fit()

sum = res.summary(xname=['Intercept', 'AFD', 'CSU', 'Die Grünen', 'Die Linke', 'Die blaue Partei', 'FDP', 'Parteilos', 'SPD', 'year'])
sum

Optimization terminated successfully.
         Current function value: 1080.235627
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:                  score   No. Observations:                13620
Model:                        Poisson   Df Residuals:                    13610
Method:                           MLE   Df Model:                            9
Date:                Thu, 19 Nov 2020   Pseudo R-squ.:                 0.04581
Time:                        13:25:45   Log-Likelihood:            -1.4713e+07
converged:                       True   LL-Null:                   -1.5419e+07
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            5.8062      0.005   1258.324      0.000       5.797       5.815
AFD                  1.1322      0.001    864.594      0.000       1.130       1.135
CSU                  0.0053      0.002      2.627      0.009       0.001       0.009
Die Grünen           0.0822      0.002     46.691      0.000       0.079       0.086
Die Linke            0.8617      0.001    594.378      0.000       0.859       0.865
Die blaue Partei     1.2066      0.003    387.958      0.000       1.201       1.213
FDP                  0.3368      0.002    210.680      0.000       0.334       0.340
Parteilos            1.5000      0.006    257.645      0.000       1.489       1.511
SPD                  0.0187      0.002     12.341      0.000       0.016       0.022
year                -0.0208      0.001    -25.717      0.000      -0.022      -0.019
====================================================================================
"""

In [29]:
print(sum.tables[1].as_text())

                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            5.8062      0.005   1258.324      0.000       5.797       5.815
AFD                  1.1322      0.001    864.594      0.000       1.130       1.135
CSU                  0.0053      0.002      2.627      0.009       0.001       0.009
Die Grünen           0.0822      0.002     46.691      0.000       0.079       0.086
Die Linke            0.8617      0.001    594.378      0.000       0.859       0.865
Die blaue Partei     1.2066      0.003    387.958      0.000       1.201       1.213
FDP                  0.3368      0.002    210.680      0.000       0.334       0.340
Parteilos            1.5000      0.006    257.645      0.000       1.489       1.511
SPD                  0.0187      0.002     12.341      0.000       0.016       0.022
year                -0.0208      0.001    -25.717      0.000     

In [99]:
LRresult = (sum.tables[1])

In [100]:
df = pd.DataFrame(LRresult)

In [101]:
df.columns = df.iloc[0]

In [14]:
df.to_text()

AttributeError: 'DataFrame' object has no attribute 'to_text'

In [4]:
import dash
import dash_table
import pandas as pd
from jupyter_dash import JupyterDash

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_usa_states.csv')

# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app = JupyterDash(__name__)

server = app.server

# app = dash.Dash(__name__)

app.layout = dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i} for i in df.columns],
    data=df.to_dict("rows"),
)

app.run_server(debug=True)

ConnectionError: HTTPConnectionPool(host='x86_64-conda-linux-gnu', port=8050): Max retries exceeded with url: /_alive_5aeb3e44-dfcc-4042-aaa1-b1823f8a29b9 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9af07b6d50>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

Exception in thread Thread-52:
Traceback (most recent call last):
  File "/home/philippy/anaconda3/envs/dsc/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/philippy/anaconda3/envs/dsc/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/philippy/anaconda3/envs/dsc/lib/python3.7/site-packages/retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "/home/philippy/anaconda3/envs/dsc/lib/python3.7/site-packages/retrying.py", line 212, in call
    raise attempt.get()
  File "/home/philippy/anaconda3/envs/dsc/lib/python3.7/site-packages/retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/home/philippy/anaconda3/envs/dsc/lib/python3.7/site-packages/six.py", line 703, in reraise
    raise value
  File "/home/philippy/anaconda3/envs/dsc/lib/python3.7/site-packages/retrying.py", line 200, in call
    attempt = At

In [103]:
print(df.to_latex())

\begin{tabular}{llllllll}
\toprule
0 &     coef    &  std err   &     z      &  P>|z|  &   [0.025   &   0.975]   \\
\midrule
0  &                                                 &        coef &    std err &          z &   P>|z| &     [0.025 &     0.975] \\
1  &                                       Intercept &      5.8062 &      0.005 &   1258.324 &   0.000 &      5.797 &      5.815 \\
2  &               C(party, Treatment('CDU'))[T.AfD] &      1.1322 &      0.001 &    864.594 &   0.000 &      1.130 &      1.135 \\
3  &               C(party, Treatment('CDU'))[T.CSU] &      0.0053 &      0.002 &      2.627 &   0.009 &      0.001 &      0.009 \\
4  &        C(party, Treatment('CDU'))[T.DIE GRÜNEN] &      0.0822 &      0.002 &     46.691 &   0.000 &      0.079 &      0.086 \\
5  &         C(party, Treatment('CDU'))[T.DIE LINKE] &      0.8617 &      0.001 &    594.378 &   0.000 &      0.859 &      0.865 \\
6  &  C(party, Treatment('CDU'))[T.Die blaue Partei] &      1.2066 &      0.003 &  

%%latex
<table>
    <tr>
        <td>
\begin{tabular}{llllllll}
\toprule
0 &     coef    &  std err   &     z      &  P>|z|  &   [0.025   &   0.975]   \\
\midrule
0  &                                                 &        coef &    std err &          z &   P>|z| &     [0.025 &     0.975] \\
1  &                                       Intercept &      5.8062 &      0.005 &   1258.324 &   0.000 &      5.797 &      5.815 \\
2  &               C(party, Treatment('CDU'))[T.AfD] &      1.1322 &      0.001 &    864.594 &   0.000 &      1.130 &      1.135 \\
3  &               C(party, Treatment('CDU'))[T.CSU] &      0.0053 &      0.002 &      2.627 &   0.009 &      0.001 &      0.009 \\
4  &        C(party, Treatment('CDU'))[T.DIE GRÜNEN] &      0.0822 &      0.002 &     46.691 &   0.000 &      0.079 &      0.086 \\
5  &         C(party, Treatment('CDU'))[T.DIE LINKE] &      0.8617 &      0.001 &    594.378 &   0.000 &      0.859 &      0.865 \\
6  &  C(party, Treatment('CDU'))[T.Die blaue Partei] &      1.2066 &      0.003 &    387.958 &   0.000 &      1.201 &      1.213 \\
7  &               C(party, Treatment('CDU'))[T.FDP] &      0.3368 &      0.002 &    210.680 &   0.000 &      0.334 &      0.340 \\
8  &         C(party, Treatment('CDU'))[T.Parteilos] &      1.5000 &      0.006 &    257.645 &   0.000 &      1.489 &      1.511 \\
9  &               C(party, Treatment('CDU'))[T.SPD] &      0.0187 &      0.002 &     12.341 &   0.000 &      0.016 &      0.022 \\
10 &                                            year &     -0.0208 &      0.001 &    -25.717 &   0.000 &     -0.022 &     -0.019 \\
\bottomrule
\end{tabular}
                    </td>
    </tr>
</table>

%%latex
\begin{tabular}{lrllr}
\toprule
{} &  Rank &         State         & Postal &  Population \\
\midrule
0  &    1  &               Alabama &   AL   &   4849377.0 \\
1  &    2  &                Alaska &   AK   &    736732.0 \\
2  &    3  &               Arizona &   AZ   &   6731484.0 \\
3  &    4  &              Arkansas &   AR   &   2966369.0 \\
4  &    5  &            California &   CA   &  38802500.0 \\
5  &    6  &              Colorado &   CO   &   5355866.0 \\
6  &    7  &           Connecticut &   CT   &   3596677.0 \\
7  &    8  &              Delaware &   DE   &    935614.0 \\
8  &    9  &  District of Columbia &   DC   &    658893.0 \\
9  &   10  &               Florida &   FL   &  19893297.0 \\
10 &   11  &               Georgia &   GA   &  10097343.0 \\
11 &   12  &                Hawaii &   HI   &   1419561.0 \\
12 &   13  &                 Idaho &   ID   &   1634464.0 \\
13 &   14  &              Illinois &   IL   &  12880580.0 \\
14 &   15  &               Indiana &   IN   &   6596855.0 \\
15 &   16  &                  Iowa &   IA   &   3107126.0 \\
16 &   17  &                Kansas &   KS   &   2904021.0 \\
17 &   18  &              Kentucky &   KY   &   4413457.0 \\
18 &   19  &             Louisiana &   LA   &   4649676.0 \\
19 &   20  &                 Maine &   ME   &   1330089.0 \\
20 &   21  &              Maryland &   MD   &   5976407.0 \\
21 &   22  &         Massachusetts &   MA   &   6745408.0 \\
22 &   23  &              Michigan &   MI   &   9909877.0 \\
23 &   24  &             Minnesota &   MN   &   5457173.0 \\
24 &   25  &           Mississippi &   MS   &   2994079.0 \\
25 &   26  &              Missouri &   MO   &   6063589.0 \\
26 &   27  &               Montana &   MT   &   1023579.0 \\
27 &   28  &              Nebraska &   NE   &   1881503.0 \\
28 &   29  &                Nevada &   NV   &   2839098.0 \\
29 &   30  &         New Hampshire &   NH   &   1326813.0 \\
30 &   31  &            New Jersey &   NJ   &   8938175.0 \\
31 &   32  &            New Mexico &   NM   &   2085572.0 \\
32 &   33  &              New York &   NY   &  19746227.0 \\
33 &   34  &        North Carolina &   NC   &   9943964.0 \\
34 &   35  &          North Dakota &   ND   &    739482.0 \\
35 &   36  &                  Ohio &   OH   &  11594163.0 \\
36 &   37  &              Oklahoma &   OK   &   3878051.0 \\
37 &   38  &                Oregon &   OR   &   3970239.0 \\
38 &   39  &          Pennsylvania &   PA   &  12787209.0 \\
39 &   40  &           Puerto Rico &   PR   &   3548397.0 \\
40 &   41  &          Rhode Island &   RI   &   1055173.0 \\
41 &   42  &        South Carolina &   SC   &   4832482.0 \\
42 &   43  &          South Dakota &   SD   &    853175.0 \\
43 &   44  &             Tennessee &   TN   &   6549352.0 \\
44 &   45  &                 Texas &   TX   &  26956958.0 \\
45 &   46  &                  Utah &   UT   &   2942902.0 \\
46 &   47  &               Vermont &   VT   &    626562.0 \\
47 &   48  &              Virginia &   VA   &   8326289.0 \\
48 &   49  &            Washington &   WA   &   7061530.0 \\
49 &   50  &         West Virginia &   WV   &   1850326.0 \\
50 &   51  &             Wisconsin &   WI   &   5757564.0 \\
51 &   52  &               Wyoming &   WY   &    584153.0 \\
\bottomrule
\end{tabular}

In [93]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_usa_states.csv')

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df.Rank, df.State, df.Postal, df.Population],
               fill_color='lavender',
               align='left'))
])

fig.show()

In [94]:
print(fig)

Figure({
    'data': [{'cells': {'align': 'left',
                        'fill': {'color': 'lavender'},
                        'values': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                   15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
                                   27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
                                   39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
                                   51, 52], ['Alabama', 'Alaska', 'Arizona',
                                   'Arkansas', 'California', 'Colorado',
                                   'Connecticut', 'Delaware', 'District of
                                   Columbia', 'Florida', 'Georgia', 'Hawaii',
                                   'Idaho', 'Illinois', 'Indiana', 'Iowa',
                                   'Kansas', 'Kentucky', 'Louisiana', 'Maine',
                                   'Maryland', 'Massachusetts', 'Michigan',
                              

In [91]:
print(df.to_latex(index=False))

\begin{tabular}{rllr}
\toprule
 Rank &         State         & Postal &  Population \\
\midrule
   1  &               Alabama &   AL   &   4849377.0 \\
   2  &                Alaska &   AK   &    736732.0 \\
   3  &               Arizona &   AZ   &   6731484.0 \\
   4  &              Arkansas &   AR   &   2966369.0 \\
   5  &            California &   CA   &  38802500.0 \\
   6  &              Colorado &   CO   &   5355866.0 \\
   7  &           Connecticut &   CT   &   3596677.0 \\
   8  &              Delaware &   DE   &    935614.0 \\
   9  &  District of Columbia &   DC   &    658893.0 \\
  10  &               Florida &   FL   &  19893297.0 \\
  11  &               Georgia &   GA   &  10097343.0 \\
  12  &                Hawaii &   HI   &   1419561.0 \\
  13  &                 Idaho &   ID   &   1634464.0 \\
  14  &              Illinois &   IL   &  12880580.0 \\
  15  &               Indiana &   IN   &   6596855.0 \\
  16  &                  Iowa &   IA   &   3107126.0 \\
  17  & 

\begin{tabular}{lllllll}
\toprule
                                                &     coef    &  std err   &     z      &  P>|z|  &   [0.025   &   0.975]   \\
\midrule
                                                &        coef &    std err &          z &   P>|z| &     [0.025 &     0.975] \\
                                      Intercept &      5.8062 &      0.005 &   1258.324 &   0.000 &      5.797 &      5.815 \\
              C(party, Treatment('CDU'))[T.AfD] &      1.1322 &      0.001 &    864.594 &   0.000 &      1.130 &      1.135 \\
              C(party, Treatment('CDU'))[T.CSU] &      0.0053 &      0.002 &      2.627 &   0.009 &      0.001 &      0.009 \\
       C(party, Treatment('CDU'))[T.DIE GRÜNEN] &      0.0822 &      0.002 &     46.691 &   0.000 &      0.079 &      0.086 \\
        C(party, Treatment('CDU'))[T.DIE LINKE] &      0.8617 &      0.001 &    594.378 &   0.000 &      0.859 &      0.865 \\
 C(party, Treatment('CDU'))[T.Die blaue Partei] &      1.2066 &      0.003 &    387.958 &   0.000 &      1.201 &      1.213 \\
              C(party, Treatment('CDU'))[T.FDP] &      0.3368 &      0.002 &    210.680 &   0.000 &      0.334 &      0.340 \\
        C(party, Treatment('CDU'))[T.Parteilos] &      1.5000 &      0.006 &    257.645 &   0.000 &      1.489 &      1.511 \\
              C(party, Treatment('CDU'))[T.SPD] &      0.0187 &      0.002 &     12.341 &   0.000 &      0.016 &      0.022 \\
                                           year &     -0.0208 &      0.001 &    -25.717 &   0.000 &     -0.022 &     -0.019 \\
\bottomrule
\end{tabular}

In [79]:
IPython.display.Latex
df.to_latex()

"\\begin{tabular}{llllllll}\n\\toprule\n0 &     coef    &  std err   &     z      &  P>|z|  &   [0.025   &   0.975]   \\\\\n\\midrule\n0  &                                                 &        coef &    std err &          z &   P>|z| &     [0.025 &     0.975] \\\\\n1  &                                       Intercept &      5.8062 &      0.005 &   1258.324 &   0.000 &      5.797 &      5.815 \\\\\n2  &               C(party, Treatment('CDU'))[T.AfD] &      1.1322 &      0.001 &    864.594 &   0.000 &      1.130 &      1.135 \\\\\n3  &               C(party, Treatment('CDU'))[T.CSU] &      0.0053 &      0.002 &      2.627 &   0.009 &      0.001 &      0.009 \\\\\n4  &        C(party, Treatment('CDU'))[T.DIE GRÜNEN] &      0.0822 &      0.002 &     46.691 &   0.000 &      0.079 &      0.086 \\\\\n5  &         C(party, Treatment('CDU'))[T.DIE LINKE] &      0.8617 &      0.001 &    594.378 &   0.000 &      0.859 &      0.865 \\\\\n6  &  C(party, Treatment('CDU'))[T.Die blaue Partei] & 

In [72]:
import IPython

pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

html_string = '''
<html>
  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
    {table}
  </body>
</html>.
'''

# OUTPUT AN HTML FILE
with open('myhtml.html', 'w') as f:
    f.write(html_string.format(table=df.to_html(classes='https://unpkg.com/papercss@1.8.1/dist/paper.css')))

# IPython.display.HTML(html_string.format(table=df.to_html(classes='mystyle.css')))
# IFrame('mythml.html', width=800, height=600)

In [61]:
df.style()

,,coef,std err,z,P>|z|,[0.025,0.975]
0,,coef,std err,z,P>|z|,[0.025,0.975]
1,Intercept,5.8062,0.005,1258.324,0.000,5.797,5.815
2,"C(party, Treatment('CDU'))[T.AfD]",1.1322,0.001,864.594,0.000,1.130,1.135
3,"C(party, Treatment('CDU'))[T.CSU]",0.0053,0.002,2.627,0.009,0.001,0.009
4,"C(party, Treatment('CDU'))[T.DIE GRÜNEN]",0.0822,0.002,46.691,0.000,0.079,0.086
5,"C(party, Treatment('CDU'))[T.DIE LINKE]",0.8617,0.001,594.378,0.000,0.859,0.865
6,"C(party, Treatment('CDU'))[T.Die blaue Partei]",1.2066,0.003,387.958,0.000,1.201,1.213
7,"C(party, Treatment('CDU'))[T.FDP]",0.3368,0.002,210.680,0.000,0.334,0.340
8,"C(party, Treatment('CDU'))[T.Parteilos]",1.5000,0.006,257.645,0.000,1.489,1.511
9,"C(party, Treatment('CDU'))[T.SPD]",0.0187,0.002,12.341,0.000,0.016,0.022


In [67]:
from IPython.display import IFrame
IFrame('myhtml.html', width=800, height=600)

In [62]:
dir(df.style)

['__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_applymap',
 '_background_gradient',
 '_bar',
 '_compute',
 '_copy',
 '_display_funcs',
 '_highlight_extrema',
 '_highlight_handler',
 '_highlight_null',
 '_repr_html_',
 '_todo',
 '_translate',
 '_update_ctx',
 'apply',
 'applymap',
 'background_gradient',
 'bar',
 'caption',
 'cell_ids',
 'clear',
 'columns',
 'ctx',
 'data',
 'env',
 'export',
 'format',
 'from_custom_template',
 'hidden_columns',
 'hidden_index',
 'hide_columns',
 'hide_index',
 'highlight_max',
 'highlight_min',
 'highlight_null',
 'index',
 'loader',
 'na_rep',
 'pipe',
 'precision',
 'render',
 'set_caption

In [56]:
from IPython.display import HTML
HTML('<style>{}</style>'.format(open('mys.css').read()))

FileNotFoundError: [Errno 2] No such file or directory: 'mys.css'

In [52]:
import IPython
IPython.display.HTML(filename='myhtml.html')

,,coef,std err,z,P>|z|,[0.025,0.975]
0,,coef,std err,z,P>|z|,[0.025,0.975]
1,Intercept,5.8062,0.005,1258.324,0.000,5.797,5.815
2,"C(party, Treatment('CDU'))[T.AfD]",1.1322,0.001,864.594,0.000,1.130,1.135
3,"C(party, Treatment('CDU'))[T.CSU]",0.0053,0.002,2.627,0.009,0.001,0.009
4,"C(party, Treatment('CDU'))[T.DIE GRÜNEN]",0.0822,0.002,46.691,0.000,0.079,0.086
5,"C(party, Treatment('CDU'))[T.DIE LINKE]",0.8617,0.001,594.378,0.000,0.859,0.865
6,"C(party, Treatment('CDU'))[T.Die blaue Partei]",1.2066,0.003,387.958,0.000,1.201,1.213
7,"C(party, Treatment('CDU'))[T.FDP]",0.3368,0.002,210.680,0.000,0.334,0.340
8,"C(party, Treatment('CDU'))[T.Parteilos]",1.5000,0.006,257.645,0.000,1.489,1.511
9,"C(party, Treatment('CDU'))[T.SPD]",0.0187,0.002,12.341,0.000,0.016,0.022


<table border="1" class="dataframe mystyle.css">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th></th>\n      <th>coef</th>\n      <th>std err</th>\n      <th>z</th>\n      <th>P&gt;|z|</th>\n      <th>[0.025</th>\n      <th>0.975]</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td></td>\n      <td>coef</td>\n      <td>std err</td>\n      <td>z</td>\n      <td>P&gt;|z|</td>\n      <td>[0.025</td>\n      <td>0.975]</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Intercept</td>\n      <td>5.8062</td>\n      <td>0.005</td>\n      <td>1258.324</td>\n      <td>0.000</td>\n      <td>5.797</td>\n      <td>5.815</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>C(party, Treatment(\'CDU\'))[T.AfD]</td>\n      <td>1.1322</td>\n      <td>0.001</td>\n      <td>864.594</td>\n      <td>0.000</td>\n      <td>1.130</td>\n      <td>1.135</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>C(party, Treatment(\'CDU\'))[T.CSU]</td>\n      <td>0.0053</td>\n      <td>0.002</td>\n      <td>2.627</td>\n      <td>0.009</td>\n      <td>0.001</td>\n      <td>0.009</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>C(party, Treatment(\'CDU\'))[T.DIE GRÜNEN]</td>\n      <td>0.0822</td>\n      <td>0.002</td>\n      <td>46.691</td>\n      <td>0.000</td>\n      <td>0.079</td>\n      <td>0.086</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>C(party, Treatment(\'CDU\'))[T.DIE LINKE]</td>\n      <td>0.8617</td>\n      <td>0.001</td>\n      <td>594.378</td>\n      <td>0.000</td>\n      <td>0.859</td>\n      <td>0.865</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>C(party, Treatment(\'CDU\'))[T.Die blaue Partei]</td>\n      <td>1.2066</td>\n      <td>0.003</td>\n      <td>387.958</td>\n      <td>0.000</td>\n      <td>1.201</td>\n      <td>1.213</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>C(party, Treatment(\'CDU\'))[T.FDP]</td>\n      <td>0.3368</td>\n      <td>0.002</td>\n      <td>210.680</td>\n      <td>0.000</td>\n      <td>0.334</td>\n      <td>0.340</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td>C(party, Treatment(\'CDU\'))[T.Parteilos]</td>\n      <td>1.5000</td>\n      <td>0.006</td>\n      <td>257.645</td>\n      <td>0.000</td>\n      <td>1.489</td>\n      <td>1.511</td>\n    </tr>\n    <tr>\n      <th>9</th>\n      <td>C(party, Treatment(\'CDU\'))[T.SPD]</td>\n      <td>0.0187</td>\n      <td>0.002</td>\n      <td>12.341</td>\n      <td>0.000</td>\n      <td>0.016</td>\n      <td>0.022</td>\n    </tr>\n    <tr>\n      <th>10</th>\n      <td>year</td>\n      <td>-0.0208</td>\n      <td>0.001</td>\n      <td>-25.717</td>\n      <td>0.000</td>\n      <td>-0.022</td>\n      <td>-0.019</td>\n    </tr>\n  </tbody>\n</table>

In [17]:
%%latex
"\\begin{center}\n\\begin{tabular}{lclc}\n\\toprule\n\\textbf{Dep. Variable:}                                 &      score       & \\textbf{  No. Observations:  } &     13620    \\\\\n\\textbf{Model:}                                         &     Poisson      & \\textbf{  Df Residuals:      } &     13610    \\\\\n\\textbf{Method:}                                        &       MLE        & \\textbf{  Df Model:          } &         9    \\\\\n\\textbf{Date:}                                          & Thu, 19 Nov 2020 & \\textbf{  Pseudo R-squ.:     } &   0.04581    \\\\\n\\textbf{Time:}                                          &     09:57:21     & \\textbf{  Log-Likelihood:    } & -1.4713e+07  \\\\\n\\textbf{converged:}                                     &       True       & \\textbf{  LL-Null:           } & -1.5419e+07  \\\\\n\\bottomrule\n\\end{tabular}\n\\begin{tabular}{lcccccc}\n                                                        & \\textbf{coef} & \\textbf{std err} & \\textbf{z} & \\textbf{P$> |$z$|$} & \\textbf{[0.025} & \\textbf{0.975]}  \\\\\n\\midrule\n\\textbf{Intercept}                                      &       5.8062  &        0.005     &  1258.324  &         0.000        &        5.797    &        5.815     \\\\\n\\textbf{C(party, Treatment('CDU'))[T.AfD]}              &       1.1322  &        0.001     &   864.594  &         0.000        &        1.130    &        1.135     \\\\\n\\textbf{C(party, Treatment('CDU'))[T.CSU]}              &       0.0053  &        0.002     &     2.627  &         0.009        &        0.001    &        0.009     \\\\\n\\textbf{C(party, Treatment('CDU'))[T.DIE GRÜNEN]}       &       0.0822  &        0.002     &    46.691  &         0.000        &        0.079    &        0.086     \\\\\n\\textbf{C(party, Treatment('CDU'))[T.DIE LINKE]}        &       0.8617  &        0.001     &   594.378  &         0.000        &        0.859    &        0.865     \\\\\n\\textbf{C(party, Treatment('CDU'))[T.Die blaue Partei]} &       1.2066  &        0.003     &   387.958  &         0.000        &        1.201    &        1.213     \\\\\n\\textbf{C(party, Treatment('CDU'))[T.FDP]}              &       0.3368  &        0.002     &   210.680  &         0.000        &        0.334    &        0.340     \\\\\n\\textbf{C(party, Treatment('CDU'))[T.Parteilos]}        &       1.5000  &        0.006     &   257.645  &         0.000        &        1.489    &        1.511     \\\\\n\\textbf{C(party, Treatment('CDU'))[T.SPD]}              &       0.0187  &        0.002     &    12.341  &         0.000        &        0.016    &        0.022     \\\\\n\\textbf{year}                                           &      -0.0208  &        0.001     &   -25.717  &         0.000        &       -0.022    &       -0.019     \\\\\n\\bottomrule\n\\end{tabular}\n%\\caption{Poisson Regression Results}\n\\end{center}"

<IPython.core.display.Latex object>

In [22]:
sum.as_csv()

"                     Poisson Regression Results                    \nDep. Variable:  ,score           ,  No. Observations:  , 13620     \nModel:          ,Poisson         ,  Df Residuals:      , 13610     \nMethod:         ,MLE             ,  Df Model:          ,     9     \nDate:           ,Thu, 19 Nov 2020,  Pseudo R-squ.:     ,0.04581    \nTime:           ,10:47:48        ,  Log-Likelihood:    ,-1.4713e+07\nconverged:      ,True            ,  LL-Null:           ,-1.5419e+07\nCovariance Type:,nonrobust       ,  LLR p-value:       , 0.000     \n                                              ,   coef   , std err ,    z    ,P>|z| ,  [0.025 ,  0.975] \nIntercept                                     ,    5.8062,    0.005, 1258.324, 0.000,    5.797,    5.815\nC(party, Treatment('CDU'))[T.AfD]             ,    1.1322,    0.001,  864.594, 0.000,    1.130,    1.135\nC(party, Treatment('CDU'))[T.CSU]             ,    0.0053,    0.002,    2.627, 0.009,    0.001,    0.009\nC(party, Treatment('CDU

In [23]:
sum.summary2()

AttributeError: 'Summary' object has no attribute 'summary2'

In [22]:
from statsmodels.iolib.table import SimpleTable

In [57]:
import pandas as pd
from io import StringIO

StringData = StringIO(f"""{sum.tables[1].as_csv()}""")
df = pd.read_csv(StringData, sep=';')

In [58]:
df

,", coef , std err , z ,P>|z| , [0.025 , 0.975]"
0,Intercept ...
1,"C(party, Treatment('CDU'))[T.AfD] ..."
2,"C(party, Treatment('CDU'))[T.CSU] ..."
3,"C(party, Treatment('CDU'))[T.DIE GRÜNEN] ..."
4,"C(party, Treatment('CDU'))[T.DIE LINKE] ..."
5,"C(party, Treatment('CDU'))[T.Die blaue Partei]..."
6,"C(party, Treatment('CDU'))[T.FDP] ..."
7,"C(party, Treatment('CDU'))[T.Parteilos] ..."
8,"C(party, Treatment('CDU'))[T.SPD] ..."
9,year ...


In [27]:
LRresult = (sum.tables[1])

In [29]:
df = pd.read_html(LRresult.as_html(), header=0)

In [30]:
df

[                                       Unnamed: 0    coef  std err         z  \
 0                                       Intercept  5.8062    0.005  1258.324   
 1               C(party, Treatment('CDU'))[T.AfD]  1.1322    0.001   864.594   
 2               C(party, Treatment('CDU'))[T.CSU]  0.0053    0.002     2.627   
 3        C(party, Treatment('CDU'))[T.DIE GRÜNEN]  0.0822    0.002    46.691   
 4         C(party, Treatment('CDU'))[T.DIE LINKE]  0.8617    0.001   594.378   
 5  C(party, Treatment('CDU'))[T.Die blaue Partei]  1.2066    0.003   387.958   
 6               C(party, Treatment('CDU'))[T.FDP]  0.3368    0.002   210.680   
 7         C(party, Treatment('CDU'))[T.Parteilos]  1.5000    0.006   257.645   
 8               C(party, Treatment('CDU'))[T.SPD]  0.0187    0.002    12.341   
 9                                            year -0.0208    0.001   -25.717   
 
    P>|z|  [0.025  0.975]  
 0  0.000   5.797   5.815  
 1  0.000   1.130   1.135  
 2  0.009   0.001   0.0

<table class="simpletable">\n<tr>\n                         <td></td>                           <th>coef</th>     <th>std err</th>      <th>z</th>      <th>P>|z|</th>  <th>[0.025</th>    <th>0.975]</th>  \n</tr>\n<tr>\n  <th>Intercept</th>                                      <td>    5.8062</td> <td>    0.005</td> <td> 1258.324</td> <td> 0.000</td> <td>    5.797</td> <td>    5.815</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.AfD]</th>              <td>    1.1322</td> <td>    0.001</td> <td>  864.594</td> <td> 0.000</td> <td>    1.130</td> <td>    1.135</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.CSU]</th>              <td>    0.0053</td> <td>    0.002</td> <td>    2.627</td> <td> 0.009</td> <td>    0.001</td> <td>    0.009</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.DIE GRÜNEN]</th>       <td>    0.0822</td> <td>    0.002</td> <td>   46.691</td> <td> 0.000</td> <td>    0.079</td> <td>    0.086</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.DIE LINKE]</th>        <td>    0.8617</td> <td>    0.001</td> <td>  594.378</td> <td> 0.000</td> <td>    0.859</td> <td>    0.865</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.Die blaue Partei]</th> <td>    1.2066</td> <td>    0.003</td> <td>  387.958</td> <td> 0.000</td> <td>    1.201</td> <td>    1.213</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.FDP]</th>              <td>    0.3368</td> <td>    0.002</td> <td>  210.680</td> <td> 0.000</td> <td>    0.334</td> <td>    0.340</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.Parteilos]</th>        <td>    1.5000</td> <td>    0.006</td> <td>  257.645</td> <td> 0.000</td> <td>    1.489</td> <td>    1.511</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.SPD]</th>              <td>    0.0187</td> <td>    0.002</td> <td>   12.341</td> <td> 0.000</td> <td>    0.016</td> <td>    0.022</td>\n</tr>\n<tr>\n  <th>year</th>                                           <td>   -0.0208</td> <td>    0.001</td> <td>  -25.717</td> <td> 0.000</td> <td>   -0.022</td> <td>   -0.019</td>\n</tr>\n</table>

In [31]:
df = pd.read_table(LRresult)

ValueError: Invalid file path or buffer object type: <class 'statsmodels.iolib.table.SimpleTable'>

In [24]:
df.columns = df.iloc[0]

AttributeError: 'list' object has no attribute 'iloc'

In [68]:
df

,0,1,2,3
0,Dep. Variable:,score,No. Observations:,13620
1,Model:,Poisson,Df Residuals:,13610
2,Method:,MLE,Df Model:,9
3,Date:,"Thu, 19 Nov 2020",Pseudo R-squ.:,0.04581
4,Time:,09:57:21,Log-Likelihood:,-1.4713e+07
5,converged:,True,LL-Null:,-1.5419e+07
6,Covariance Type:,nonrobust,LLR p-value:,0.000


In [66]:
LRresult.to_html()

AttributeError: 'SimpleTable' object has no attribute 'to_html'

In [46]:
sum.as_csv()

"                     Poisson Regression Results                    \nDep. Variable:  ,score           ,  No. Observations:  , 13620     \nModel:          ,Poisson         ,  Df Residuals:      , 13610     \nMethod:         ,MLE             ,  Df Model:          ,     9     \nDate:           ,Thu, 19 Nov 2020,  Pseudo R-squ.:     ,0.04581    \nTime:           ,09:57:21        ,  Log-Likelihood:    ,-1.4713e+07\nconverged:      ,True            ,  LL-Null:           ,-1.5419e+07\nCovariance Type:,nonrobust       ,  LLR p-value:       , 0.000     \n                                              ,   coef   , std err ,    z    ,P>|z| ,  [0.025 ,  0.975] \nIntercept                                     ,    5.8062,    0.005, 1258.324, 0.000,    5.797,    5.815\nC(party, Treatment('CDU'))[T.AfD]             ,    1.1322,    0.001,  864.594, 0.000,    1.130,    1.135\nC(party, Treatment('CDU'))[T.CSU]             ,    0.0053,    0.002,    2.627, 0.009,    0.001,    0.009\nC(party, Treatment('CDU

In [52]:
results_as_html = sum.tables[1].as_html()
pd.read_html(results_as_html, header=0, index_col=0)[0]

ImportError: lxml not found, please install it

In [55]:
SimpleTable(sum.tables[0].as_csv())

""
""
""
""
""
""
""
""
""
""
""


In [31]:
sum.as_html()

'<table class="simpletable">\n<caption>Poisson Regression Results</caption>\n<tr>\n  <th>Dep. Variable:</th>         <td>score</td>      <th>  No. Observations:  </th>   <td> 13620</td>   \n</tr>\n<tr>\n  <th>Model:</th>                <td>Poisson</td>     <th>  Df Residuals:      </th>   <td> 13610</td>   \n</tr>\n<tr>\n  <th>Method:</th>                 <td>MLE</td>       <th>  Df Model:          </th>   <td>     9</td>   \n</tr>\n<tr>\n  <th>Date:</th>            <td>Thu, 19 Nov 2020</td> <th>  Pseudo R-squ.:     </th>   <td>0.04581</td>  \n</tr>\n<tr>\n  <th>Time:</th>                <td>09:57:21</td>     <th>  Log-Likelihood:    </th> <td>-1.4713e+07</td>\n</tr>\n<tr>\n  <th>converged:</th>             <td>True</td>       <th>  LL-Null:           </th> <td>-1.5419e+07</td>\n</tr>\n<tr>\n  <th>Covariance Type:</th>     <td>nonrobust</td>    <th>  LLR p-value:       </th>   <td> 0.000</td>   \n</tr>\n</table>\n<table class="simpletable">\n<tr>\n                         <td></td>    

<table class="simpletable">\n<caption>Poisson Regression Results</caption>\n<tr>\n  <th>Dep. Variable:</th>         <td>score</td>      <th>  No. Observations:  </th>   <td> 13620</td>   \n</tr>\n<tr>\n  <th>Model:</th>                <td>Poisson</td>     <th>  Df Residuals:      </th>   <td> 13610</td>   \n</tr>\n<tr>\n  <th>Method:</th>                 <td>MLE</td>       <th>  Df Model:          </th>   <td>     9</td>   \n</tr>\n<tr>\n  <th>Date:</th>            <td>Thu, 19 Nov 2020</td> <th>  Pseudo R-squ.:     </th>   <td>0.04581</td>  \n</tr>\n<tr>\n  <th>Time:</th>                <td>09:57:21</td>     <th>  Log-Likelihood:    </th> <td>-1.4713e+07</td>\n</tr>\n<tr>\n  <th>converged:</th>             <td>True</td>       <th>  LL-Null:           </th> <td>-1.5419e+07</td>\n</tr>\n<tr>\n  <th>Covariance Type:</th>     <td>nonrobust</td>    <th>  LLR p-value:       </th>   <td> 0.000</td>   \n</tr>\n</table>\n<table class="simpletable">\n<tr>\n                         <td></td>                           <th>coef</th>     <th>std err</th>      <th>z</th>      <th>P>|z|</th>  <th>[0.025</th>    <th>0.975]</th>  \n</tr>\n<tr>\n  <th>Intercept</th>                                      <td>    5.8062</td> <td>    0.005</td> <td> 1258.324</td> <td> 0.000</td> <td>    5.797</td> <td>    5.815</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.AfD]</th>              <td>    1.1322</td> <td>    0.001</td> <td>  864.594</td> <td> 0.000</td> <td>    1.130</td> <td>    1.135</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.CSU]</th>              <td>    0.0053</td> <td>    0.002</td> <td>    2.627</td> <td> 0.009</td> <td>    0.001</td> <td>    0.009</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.DIE GRÜNEN]</th>       <td>    0.0822</td> <td>    0.002</td> <td>   46.691</td> <td> 0.000</td> <td>    0.079</td> <td>    0.086</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.DIE LINKE]</th>        <td>    0.8617</td> <td>    0.001</td> <td>  594.378</td> <td> 0.000</td> <td>    0.859</td> <td>    0.865</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.Die blaue Partei]</th> <td>    1.2066</td> <td>    0.003</td> <td>  387.958</td> <td> 0.000</td> <td>    1.201</td> <td>    1.213</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.FDP]</th>              <td>    0.3368</td> <td>    0.002</td> <td>  210.680</td> <td> 0.000</td> <td>    0.334</td> <td>    0.340</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.Parteilos]</th>        <td>    1.5000</td> <td>    0.006</td> <td>  257.645</td> <td> 0.000</td> <td>    1.489</td> <td>    1.511</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.SPD]</th>              <td>    0.0187</td> <td>    0.002</td> <td>   12.341</td> <td> 0.000</td> <td>    0.016</td> <td>    0.022</td>\n</tr>\n<tr>\n  <th>year</th>                                           <td>   -0.0208</td> <td>    0.001</td> <td>  -25.717</td> <td> 0.000</td> <td>   -0.022</td> <td>   -0.019</td>\n</tr>\n</table>

In [36]:
%%HTML
<style>
.horizontal {flex-direction: row}
</style>

<table class="simpletable">\n<caption>Poisson Regression Results</caption>\n<tr>\n  <th>Dep. Variable:</th>         <td>score</td>      <th>  No. Observations:  </th>   <td> 13620</td>   \n</tr>\n<tr>\n  <th>Model:</th>                <td>Poisson</td>     <th>  Df Residuals:      </th>   <td> 13610</td>   \n</tr>\n<tr>\n  <th>Method:</th>                 <td>MLE</td>       <th>  Df Model:          </th>   <td>     9</td>   \n</tr>\n<tr>\n  <th>Date:</th>            <td>Thu, 19 Nov 2020</td> <th>  Pseudo R-squ.:     </th>   <td>0.04581</td>  \n</tr>\n<tr>\n  <th>Time:</th>                <td>09:57:21</td>     <th>  Log-Likelihood:    </th> <td>-1.4713e+07</td>\n</tr>\n<tr>\n  <th>converged:</th>             <td>True</td>       <th>  LL-Null:           </th> <td>-1.5419e+07</td>\n</tr>\n<tr>\n  <th>Covariance Type:</th>     <td>nonrobust</td>    <th>  LLR p-value:       </th>   <td> 0.000</td>   \n</tr>\n</table>\n<table class="simpletable">\n<tr>\n                         <td></td>                           <th>coef</th>     <th>std err</th>      <th>z</th>      <th>P>|z|</th>  <th>[0.025</th>    <th>0.975]</th>  \n</tr>\n<tr>\n  <th>Intercept</th>                                      <td>    5.8062</td> <td>    0.005</td> <td> 1258.324</td> <td> 0.000</td> <td>    5.797</td> <td>    5.815</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.AfD]</th>              <td>    1.1322</td> <td>    0.001</td> <td>  864.594</td> <td> 0.000</td> <td>    1.130</td> <td>    1.135</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.CSU]</th>              <td>    0.0053</td> <td>    0.002</td> <td>    2.627</td> <td> 0.009</td> <td>    0.001</td> <td>    0.009</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.DIE GRÜNEN]</th>       <td>    0.0822</td> <td>    0.002</td> <td>   46.691</td> <td> 0.000</td> <td>    0.079</td> <td>    0.086</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.DIE LINKE]</th>        <td>    0.8617</td> <td>    0.001</td> <td>  594.378</td> <td> 0.000</td> <td>    0.859</td> <td>    0.865</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.Die blaue Partei]</th> <td>    1.2066</td> <td>    0.003</td> <td>  387.958</td> <td> 0.000</td> <td>    1.201</td> <td>    1.213</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.FDP]</th>              <td>    0.3368</td> <td>    0.002</td> <td>  210.680</td> <td> 0.000</td> <td>    0.334</td> <td>    0.340</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.Parteilos]</th>        <td>    1.5000</td> <td>    0.006</td> <td>  257.645</td> <td> 0.000</td> <td>    1.489</td> <td>    1.511</td>\n</tr>\n<tr>\n  <th>C(party, Treatment(\'CDU\'))[T.SPD]</th>              <td>    0.0187</td> <td>    0.002</td> <td>   12.341</td> <td> 0.000</td> <td>    0.016</td> <td>    0.022</td>\n</tr>\n<tr>\n  <th>year</th>                                           <td>   -0.0208</td> <td>    0.001</td> <td>  -25.717</td> <td> 0.000</td> <td>   -0.022</td> <td>   -0.019</td>\n</tr>\n</table>

In [40]:
import pandas as pd
import numpy as np
from IPython.display import HTML

def hover(hover_color="#add8e6"):
    return dict(selector="tbody tr:hover",
            props=[("background-color", "%s" % hover_color)])

style = dict(selector=" ", 
         props=[("margin","0"),
                ("font-family",'"Helvetica", "Arial", sans-serif'),
                ("border-collapse", "collapse"),
                ("border","none"),
                ("border", "2px solid #ccf")
                   ]),

    #header color - optional
    dict(selector="thead", 
         props=[("background-color","#cc8484")
               ]),

    #background shading
    dict(selector="tbody tr:nth-child(even)",
         props=[("background-color", "#fff")]),
    dict(selector="tbody tr:nth-child(odd)",
         props=[("background-color", "#eee")]),

    #cell spacing
    dict(selector="td", 
         props=[("padding", ".5em")]),

    #header cell properties
    dict(selector="th", 
         props=[("font-size", "125%"),
                ("text-align", "center")]),

    #caption placement
    dict(selector="caption", 
         props=[("caption-side", "bottom")]),

    #render hover last to override background-color
    hover()
html = (df.style.set_table_styles(styles)
      .set_caption("Hover to highlight."))
html

IndentationError: unexpected indent (<ipython-input-40-3a1f300ee232>, line 18)